# Custom Linear Regression Demo

This notebook demonstrates the usage of the `LinearRegressionCustom` class, which implements linear regression with support for L1 and L2 regularization, early stopping, and gradient descent optimization.

We will:

- Load a housing dataset
- Preprocess features and target variables
- Train our custom linear regression model
- Visualize training loss and regression fit
- Compare predictions with scikit-learn's LinearRegression implementation

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from linear_regression_custom import LinearRegressionCustom, lin_regplot

# Ensure plots appear inline in the notebook
%matplotlib inline

## Load and Prepare Dataset

We use the Ames Housing dataset focusing on a few relevant features for simplicity.

In [ ]:
# Define columns of interest and load dataset
columns = ['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF', 'SalePrice']
df = pd.read_csv('http://jse.amstat.org/v19n3/decock/AmesHousing.txt', sep='\t', usecols=columns)

# Feature and target extraction
X = df[['Gr Liv Area']].values      # Living area square footage
y = df['SalePrice'].values          # House price

# Standardize features and targets for better convergence
sc_x = StandardScaler()
sc_y = StandardScaler()

X_std = sc_x.fit_transform(X)
y_std = sc_y.fit_transform(y[:, np.newaxis]).flatten()

## Train Custom Linear Regression Model

We train using standardized data to allow the gradient descent to converge smoothly.


In [ ]:
# Initialize and fit the custom linear regression model
lr = LinearRegressionCustom(n_iter=40, learning_rate=0.1, C=1000, penalty='l2', early_stopping=5)
lr.fit(X_std, y_std)

print(f"Model parameters: weights = {lr.w_}, bias = {lr.b_}")
print(f"Final training loss (MSE): {lr.losses_[-1]:.4f}")

## Plot Training Loss Over Epochs

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(lr.losses_, label='Training Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

## Visualize Regression Fit on Standardized Data

In [ ]:
plt.figure(figsize=(6,4))
lin_regplot(X_std, y_std, lr)
plt.show()

## Compare with Scikit-learn's LinearRegression

We also train scikit-learn's implementation on the raw data to compare weights and predictions.

In [ ]:
# Fit scikit-learn model on original data
sklr = LinearRegression()
sklr.fit(X, y)

# Predict price for a house with 2500 sqft living area
house_area = np.array([[2500]])

# Custom model prediction (scaled back to original units)
house_area_std = sc_x.transform(house_area)
price_std_pred = lr.predict(house_area_std)
price_pred = sc_y.inverse_transform(price_std_pred.reshape(-1,1))

# sklearn prediction
sk_price_pred = sklr.predict(house_area)

print(f"Custom model predicted sale price: ${price_pred.flatten()[0]:,.2f}")
print(f"Sklearn model predicted sale price: ${sk_price_pred[0]:,.2f}")

print(f"Custom model weights: {lr.w_}, bias: {lr.b_}")
print(f"Sklearn model weights: {sklr.coef_}, bias: {sklr.intercept_}"